# * Import : "Metric Master List"

## Parameter

In [1]:
import os
import glob
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import FN_CFW as fn

# Input parameter
src_dir = 'data'
src_file = 'Metric Master List'
src_sheet = 'DIM_METRIC_DATA'
curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Import Rawdata

In [2]:
data_src = f'{src_dir}/{src_file}.xlsx'
src_df = pd.read_excel(data_src, sheet_name=src_sheet, index_col=None) 
# , skiprows=2, usecols='N:AD'
src_df.head(3)

,METRIC_CD,METRIC_NAME,METRIC_ID,COMP_CD,CENTER,AGG_TYPE,RR_IND,CONS_IND,METRIC_GRP,METRIC_SUB_GRP,...,Network,Sales,Experience,Mobile,Prepaid,Postpaid,TOL,TVS,TDG,B2B
0,B0R000100,Total Revenue,B0R000100,ALL,Revenue,S,1.0,N,Revenue,Revenue,...,No,Yes,No,No,No,No,No,No,No,No
1,B0R00010001,Total Inflow M1,B0R00010001,ALL,Sales,S,1.0,S,Sales,Sales,...,No,Yes,No,No,No,No,No,No,No,No
2,B0R00010001AA,Total Inflow M1 : Account Executive,B0R00010001AA,ALL,Sales,S,0.0,N,Sales,Sales,...,No,Yes,No,No,No,No,No,No,No,No


In [19]:
cols = ['METRIC_CD', 'METRIC_NAME', 'METRIC_ID', 'COMP_CD', 'CENTER', 'AGG_TYPE', 'RR_IND', 'CONS_IND', 'METRIC_GRP', 'METRIC_SUB_GRP', 'PRODUCT_GRP', 'GRY_IND', 'UOM', 'DEF', 'CAL', 
        'CORPORATE', 'MICRO_ORG', 'METRIC_TYPE', 'RADAR', 'KPI_TYPE', 'LIST', 'REMARK', 
        'COR_EQUAL_NAT', 'NATIONWIDE', 'REGION', 'PROVINCE', 'CLUSTER', 'DISTRICT', 'SUBDISTRICT', 'Data Source']

# tmp_df = src_df[cols]
tmp_df = src_df
tmp_df['MY_GROUP'] = tmp_df.apply(lambda x: fn.my_metric_group(str(x['METRIC_GRP']), str(x['METRIC_CD']), str(x['METRIC_NAME'])), axis=1)
# tmp_df['MY_GROUP'].unique()

# Filter
tmp_df = tmp_df.loc[tmp_df['CENTER'].isin(['Revenue', 'Sales', 'Profitability'])]
tmp_df = tmp_df.loc[~tmp_df['METRIC_CD'].str.contains('[0-9]C$|[0-9]H$|[0-9]MCOM$|[0-9]CUS$')]
tmp_df = tmp_df.loc[~tmp_df['METRIC_NAME'].str.contains('\(Customer Location\)')]
tmp_df = tmp_df.loc[~tmp_df['METRIC_CD'].str.contains('[0-9]A[A-K]$')]
# tmp_df = tmp_df.loc[tmp_df['METRIC_CD'].str.contains('CORP$|GEO$')]
# tmp_df = tmp_df.loc[~tmp_df['METRIC_NAME'].str.contains('\(Geo\)')]
# tmp_df = tmp_df.loc[tmp_df['PRODUCT_GRP'].isin(['All Services', 'TOL'])]
# tmp_df = tmp_df.loc[tmp_df['PRODUCT_GRP']=='All Services']
# tmp_df = tmp_df.loc[tmp_df['PRODUCT_GRP']=='Prepaid']
# tmp_df = tmp_df.loc[tmp_df['METRIC_GRP']=='Revenue']
tmp_df = tmp_df.loc[tmp_df['METRIC_GRP'].isin(['Revenue', 'Sales', 'Subs'])]
tmp_df = tmp_df.loc[tmp_df['MY_GROUP'].isin(['Revenue', 'Inflow M1', 'Gross Adds', 'Inflow M2', '%Conversion', 'GA ARPU/RC', 'Active Subs', 'ARPU', 'New/Existing'])]
# tmp_df = tmp_df.loc[tmp_df['MY_GROUP']=='Revenue']

# tmp_df = tmp_df.loc[tmp_df['METRIC_GRP']=='Revenue']
# tmp_df = tmp_df.loc[tmp_df['METRIC_GRP']=='Sales']
# tmp_df = tmp_df.loc[tmp_df['METRIC_GRP']=='Subs']
# tmp_df = tmp_df.loc[tmp_df['METRIC_NAME'].str.contains('TDG')]

tmp_df = tmp_df.sort_values(by=['CENTER', 'METRIC_GRP', 'PRODUCT_GRP', 'COMP_CD', 'METRIC_CD']) #, 'MY_GROUP'
# src_df = src_df.replace(np.nan, None)
tmp_df = tmp_df.reset_index(drop=True)

tmp_df#.tail()

,METRIC_CD,METRIC_NAME,METRIC_ID,COMP_CD,CENTER,AGG_TYPE,RR_IND,CONS_IND,METRIC_GRP,METRIC_SUB_GRP,...,Sales,Experience,Mobile,Prepaid,Postpaid,TOL,TVS,TDG,B2B,MY_GROUP
0,TB0R000100,Total Revenue : TRUE (Geo),B0R000100,True,Revenue,S,1.0,N,Revenue,Revenue,...,Yes,No,No,No,No,No,No,No,No,Revenue
1,TB0R000100CORP,Total Revenue : TRUE,B0R000100CORP,True,Revenue,S,1.0,N,Revenue,Revenue,...,Yes,No,No,No,No,No,No,No,No,Revenue
2,B0R000100,Total Revenue,B0R000100,ALL,Revenue,S,1.0,N,Revenue,Revenue,...,Yes,No,No,No,No,No,No,No,No,Revenue
3,B0R000100GEO,Total Revenue (Geo),B0R000100GEO,ALL,Revenue,S,1.0,N,Revenue,Revenue,...,Yes,No,No,No,No,No,No,No,No,Revenue
4,DB0R000100,Total Revenue : DTAC,B0R000100,DTAC,Revenue,S,1.0,N,Revenue,Revenue,...,Yes,No,No,No,No,No,No,No,No,Revenue
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,TB4R001700,TVS Now Inflow M1,TB4R001700,TRUE,Sales,S,1.0,N,Sales,Sales,...,Yes,No,No,No,No,No,Yes,No,No,Inflow M1
167,TB4S000100,TVS Gross Adds,TB4S000100,TRUE,Sales,S,1.0,N,Sales,Sales,...,Yes,No,No,No,No,No,Yes,No,No,Gross Adds
168,TB4S000100GEO,TVS Gross Adds (Geo),TB4S000100GEO,TRUE,Sales,S,1.0,N,Sales,Sales,...,Yes,No,No,No,No,No,Yes,No,No,Gross Adds
169,TB4S001300,TVS Gross Adds (Install Location),TB4S001300,TRUE,Sales,S,1.0,N,Sales,Sales,...,Yes,No,No,No,No,No,Yes,No,No,Gross Adds


In [9]:
# tmp_df = src_df[cols]
# tmp_df = tmp_df.loc[tmp_df['METRIC_ID']=='B0R000100']
# tmp_df = tmp_df.reset_index(drop=True)
# tmp_df[['METRIC_CD', 'METRIC_NAME', 'METRIC_ID', 'COMP_CD', 'CENTER', 'AGG_TYPE', 'RR_IND', 'CONS_IND', 'METRIC_GRP', 'METRIC_SUB_GRP', 'PRODUCT_GRP', 'GRY_IND']]

In [10]:
# tmp_df = src_df[cols]
# tmp_df = tmp_df.loc[tmp_df['METRIC_ID']=='B0R000100GEO']
# tmp_df = tmp_df.reset_index(drop=True)
# tmp_df[['METRIC_CD', 'METRIC_NAME', 'METRIC_ID', 'COMP_CD', 'CENTER', 'AGG_TYPE', 'RR_IND', 'CONS_IND', 'METRIC_GRP', 'METRIC_SUB_GRP', 'PRODUCT_GRP', 'GRY_IND']]

## Generate Output file

In [20]:
# to Excel file

op_dir = 'temp'
op_file = 'TMP_Metric Master List.xlsx'

# tmp_df.to_excel(f'{op_dir}/{op_file}', sheet_name='Data', index=False)
# print(f'\n  -> Generate "{op_file}" successfully')


  -> Generate "TMP_Metric Master List.xlsx" successfully
